In [3]:
import unittest
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

### First Strategy
The maximum possible number of chunks i n, where n is the length of the array. If the array is already sorted this will be the answer. Otherwise for any given chunk in the array it should be merged to the right if its min is less than the max of everything to the right, and merged to the left if its min is less than the max of the left. We start with n chunks and work from right to left successively merging when needed

In [13]:
class Solution:
    def maxChunksToSorted(self, arr) -> int:
        # start with as many chunks as there are elements each of length 1
        partitions = [(i, i+1) for i in range(len(arr))]
        for i in range(len(partitions)-2, 0, -1):
            p = partitions[i]
            min_current = min(arr[p[0]:p[1]])
            max_left = max(arr[partitions[0][0]:partitions[i-1][0]])
            print(min_current)
            print(max_left)
            if min_current < max_left:
                # current partition minimum less than left side maximum, merge current into left
                pnew = ()
            if max(arr[p[0]:p[1]]) > min([arr[partitions[i+1][0]:partitions[-1][0]]]):
                # current partition maximum greater than right side minimum
                pnew = (p[0], partitions[i+1][-1])
                partitions[i] = pnew
                partitions.pop(i+1)
                
        return len(partitions)
                

In [14]:
np.random.seed(42)
testlist = list(np.random.randint(0, 100, 20))
s = Solution()
s.maxChunksToSorted(testlist)

29
99
87
99
1
99


C:\Users\azdoh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()